<a href="https://colab.research.google.com/github/AndreiAf02/Job_Satisfaction_Burnout/blob/main/Job_Satisfaction_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import math
from scipy import stats
from scipy.stats import mannwhitneyu

In [2]:
def bootstrap_analysis(bootstrap_path_values, Working_condition_cols):
  mean_results = pd.DataFrame(columns = ['Path']+Working_condition_cols)
  lbound_results = pd.DataFrame(columns = ['Path']+Working_condition_cols)
  ubound_results = pd.DataFrame(columns = ['Path']+Working_condition_cols)
  pbci_results = pd.DataFrame(columns = ['Path']+Working_condition_cols)
  pbci_90_results = pd.DataFrame(columns = ['Path']+Working_condition_cols)

  for path in bootstrap_path_values['Path'].unique().tolist():
    data = bootstrap_path_values[bootstrap_path_values['Path']==path].reset_index(drop=True)
    mean_vals = [path]
    lbound_vals = [path]
    ubound_vals = [path]
    pbci_vals = [path]
    pbci_90_vals = [path]

    for col in Working_condition_cols:
      mean_vals.append(data[col].mean())
      ## Calculating the lower and upper bounds of the Percentile Bootstrap Confidence Interval (PBCI):
      lbound = np.percentile(data[col], 2.5)
      ubound = np.percentile(data[col], 97.5)
      if lbound > 0. or ubound < 0.:
        sig = ' *'
      else:
        sig = ''
      lbound_vals.append(lbound)
      ubound_vals.append(ubound)
      pbci_vals.append('['+str(round(lbound,4))+', '+str(round(ubound,4))+']'+sig)
      pbci_90_vals.append('['+str(round(np.percentile(data[col], 5.),4))+', '+str(round(np.percentile(data[col], 95),4))+']')

    mean_results.loc[len(mean_results)] = mean_vals
    lbound_results.loc[len(lbound_results)] = lbound_vals
    ubound_results.loc[len(ubound_results)] = ubound_vals
    pbci_results.loc[len(pbci_results)] = pbci_vals
    pbci_90_results.loc[len(pbci_90_results)] = pbci_90_vals



  return mean_results, lbound_results, ubound_results, pbci_results, pbci_90_results

In [40]:
def Bayesian_analysis(result_posterior_samples, factor_unique, dir_path_unique):
  path_ci = pd.DataFrame()
  path_ci_mean = pd.DataFrame()
  path_prob = pd.DataFrame()

  path_ci['Factor'] = factor_unique
  path_prob['Factor'] = factor_unique
  path_ci_mean['Factor'] = factor_unique

  med_ci_vals = []
  med_prob_vals = []
  mod_ci_vals = []
  mod_prob_vals = []
  dp_ci_vals = []
  dp_prob_vals = []

  med_ci_vals_mean = []
  mod_ci_vals_mean = []
  dp_ci_vals_mean = []

  for i in range(len(factor_unique)):

    col = factor_unique[i]
    ## Finding the Mediation Credibility Intervals and probabilities:
    data = result_posterior_samples[f'Indirect effect ({col})']
    lbound = np.percentile(data, 2.5)
    ubound = np.percentile(data, 97.5)
    probability_of_positivity = np.mean(data > 0)
    if lbound >0 or ubound < 0:
      sig = ' *'
    else:
      sig = ''
    med_ci_vals.append(f'[{lbound:.5f}, {ubound:.5f}]'+sig)
    med_prob_vals.append(probability_of_positivity)

    lbound_mean = data.mean() - 1.96 * data.std()/np.sqrt(len(data))
    ubound_mean = data.mean() + 1.96 * data.std()/np.sqrt(len(data))
    med_ci_vals_mean.append(f'[{lbound_mean:.5f}, {ubound_mean:.5f}]'+sig)



    ## Finding the Moderation Credibility Intervals and probabilities:
    data_mod = result_posterior_samples[f'Moderation effect ({col})']
    lbound_mod = np.percentile(data_mod, 2.5)
    ubound_mod = np.percentile(data_mod, 97.5)
    probability_of_positivity_mod = np.mean(data_mod > 0)
    if lbound_mod >0 or ubound_mod < 0:
      sig_mod = ' *'
    else:
      sig_mod = ''
    mod_ci_vals.append(f'[{lbound_mod:.5f}, {ubound_mod:.5f}]'+sig_mod)
    mod_prob_vals.append(probability_of_positivity_mod)

    lbound_mod_mean = data_mod.mean() - 1.96 * data_mod.std()/np.sqrt(len(data_mod))
    ubound_mod_mean = data.mean() + 1.96 * data.std()/np.sqrt(len(data))
    mod_ci_vals_mean.append(f'[{lbound_mod_mean:.5f}, {ubound_mod_mean:.5f}]'+sig_mod)


    ## Finding the Direct Pathway Credibility Intervals and probabilities:
    data_dp = result_posterior_samples[dir_path_unique[i]]
    lbound_dp = np.percentile(data_dp, 2.5)
    ubound_dp = np.percentile(data_dp, 97.5)
    probability_of_positivity_dp = np.mean(data_dp > 0)
    if lbound_dp >0 or ubound_dp < 0:
      sig_dp = ' *'
    else:
      sig_dp = ''
    dp_ci_vals.append(f'[{lbound_dp:.5f}, {ubound_dp:.5f}]'+sig_dp)
    dp_prob_vals.append(probability_of_positivity_dp)

    lbound_dp_mean = data_dp.mean() - 1.96 * data_dp.std()/np.sqrt(len(data_dp))
    ubound_dp_mean = data_dp.mean() + 1.96 * data_dp.std()/np.sqrt(len(data_dp))
    dp_ci_vals_mean.append(f'[{lbound_dp_mean:.5f}, {ubound_dp_mean:.5f}]'+sig_dp)


  path_ci['Mediation'] = med_ci_vals
  path_prob['Mediation'] = med_prob_vals
  path_ci['Moderation'] = mod_ci_vals
  path_prob['Moderation'] = mod_prob_vals
  path_ci['Direct Path'] = dp_ci_vals
  path_prob['Direct Path'] = dp_prob_vals

  path_ci_mean['Mediation'] = med_ci_vals_mean
  path_ci_mean['Moderation'] = mod_ci_vals_mean
  path_ci_mean['Direct Path'] = dp_ci_vals_mean

  return path_ci, path_prob, path_ci_mean

In [4]:
def Bayesian_analysis_no_mod(result_posterior_samples, factor_unique, dir_path_unique):
  path_ci = pd.DataFrame()
  path_prob = pd.DataFrame()

  path_ci['Factor'] = factor_unique
  path_prob['Factor'] = factor_unique

  med_ci_vals = []
  med_prob_vals = []
  mod_ci_vals = []
  mod_prob_vals = []
  dp_ci_vals = []
  dp_prob_vals = []

  for i in range(len(factor_unique)):

    col = factor_unique[i]
    ## Finding the Mediation Credibility Intervals and probabilities:
    data = result_posterior_samples[f'Indirect effect ({col})']
    lbound = np.percentile(data, 2.5)
    ubound = np.percentile(data, 97.5)
    probability_of_positivity = np.mean(data > 0)
    if lbound >0 or ubound < 0:
      sig = ' *'
    else:
      sig = ''
    med_ci_vals.append(f'[{lbound:.5f}, {ubound:.5f}]'+sig)
    med_prob_vals.append(probability_of_positivity)


    ## Finding the Direct Pathway Credibility Intervals and probabilities:
    data_dp = result_posterior_samples[dir_path_unique[i]]
    lbound_dp = np.percentile(data_dp, 2.5)
    ubound_dp = np.percentile(data_dp, 97.5)
    probability_of_positivity_dp = np.mean(data_dp > 0)
    if lbound_dp >0 or ubound_dp < 0:
      sig_dp = ' *'
    else:
      sig_dp = ''
    dp_ci_vals.append(f'[{lbound_dp:.5f}, {ubound_dp:.5f}]'+sig_dp)
    dp_prob_vals.append(probability_of_positivity_dp)


  path_ci['Mediation'] = med_ci_vals
  path_prob['Mediation'] = med_prob_vals
  path_ci['Direct Path'] = dp_ci_vals
  path_prob['Direct Path'] = dp_prob_vals

  return path_ci, path_prob

## Czech Republic:
**Bootstrap:**

In [5]:
path_to_bootstrap = 'https://raw.githubusercontent.com/AndreiAf02/Job_Satisfaction_Burnout/refs/heads/main/Bootstrap/CZ/'

## Path values with no moderation:
bootstrap_vals_no_mod_pos_3 = pd.read_csv(path_to_bootstrap+'CZ_bootstrap_path_values_no_mod_position_3.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_no_mod_pos_3['Position'] = 3.
bootstrap_vals_no_mod_pos_4 = pd.read_csv(path_to_bootstrap+'CZ_bootstrap_path_values_no_mod_position_4.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_no_mod_pos_4['Position'] = 4.
bootstrap_vals_no_mod_pos_5 = pd.read_csv(path_to_bootstrap+'CZ_bootstrap_path_values_no_mod_position_5.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_no_mod_pos_5['Position'] = 5.
bootstrap_vals_no_mod_pos_6 = pd.read_csv(path_to_bootstrap+'CZ_bootstrap_path_values_no_mod_position_6.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_no_mod_pos_6['Position'] = 6.

## Path values with moderation:
bootstrap_vals_pos_3 = pd.read_csv(path_to_bootstrap+'CZ_bootstrap_path_values_position_3.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_pos_3['Position'] = 3.
bootstrap_vals_pos_4 = pd.read_csv(path_to_bootstrap+'CZ_bootstrap_path_values_position_4.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_pos_4['Position'] = 4.
bootstrap_vals_pos_5 = pd.read_csv(path_to_bootstrap+'CZ_bootstrap_path_values_position_5.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_pos_5['Position'] = 5.
bootstrap_vals_pos_6 = pd.read_csv(path_to_bootstrap+'CZ_bootstrap_path_values_position_6.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_pos_6['Position'] = 6.




In [6]:
bootstrap_vals_no_mod_pos_3

,Bootstrap_nr,Path,Salary,Effort,Policy_influence,Teaching_percent,Research_percent,Academic_Resources,Perceived_Autonomy,DoF,...,CFI,GFI,AGFI,NFI,TLI,RMSEA,AIC,BIC,LogLik,Position
0,0,Mediation,-0.001130,-0.000797,0.004235,-0.000562,-0.000899,0.020858,-0.000439,28.0,...,1.151834,1.000000,1.000000,1.000000,1.233174,0.000000,33.999999,72.703323,2.755053e-07,3.0
1,0,Direct Path,-0.006148,0.005930,-0.052816,-0.002966,-0.006086,0.311291,0.397314,28.0,...,1.151834,1.000000,1.000000,1.000000,1.233174,0.000000,33.999999,72.703323,2.755053e-07,3.0
2,1,Mediation,0.006756,-0.001507,-0.038026,0.001365,0.001380,0.027286,0.027807,28.0,...,0.800997,0.723859,0.575926,0.723859,0.694389,0.144660,32.066617,70.769941,9.666917e-01,3.0
3,1,Direct Path,0.026954,0.003077,0.194853,-0.003903,-0.009183,0.387597,0.313352,28.0,...,0.800997,0.723859,0.575926,0.723859,0.694389,0.144660,32.066617,70.769941,9.666917e-01,3.0
4,2,Mediation,-0.008592,-0.001652,-0.015295,-0.000224,-0.000874,0.008115,0.038858,28.0,...,0.859687,0.765613,0.640049,0.765613,0.784520,0.115955,32.479723,71.183047,7.601383e-01,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,1997,Direct Path,0.057162,0.002172,-0.152243,-0.000192,-0.007716,0.320129,0.350899,28.0,...,0.751420,0.681317,0.510594,0.681317,0.618253,0.159688,31.814047,70.517371,1.092977e+00,3.0
3996,1998,Mediation,-0.000400,-0.000284,-0.007577,0.000056,0.000058,0.002692,0.006318,28.0,...,0.647326,0.579745,0.354609,0.579745,0.458393,0.161444,31.782906,70.486230,1.108547e+00,3.0
3997,1998,Direct Path,0.068411,0.000415,0.006469,-0.001855,-0.001824,0.345606,0.036366,28.0,...,0.647326,0.579745,0.354609,0.579745,0.458393,0.161444,31.782906,70.486230,1.108547e+00,3.0
3998,1999,Mediation,-0.002692,-0.000750,-0.005051,0.000466,-0.000226,0.026953,0.015448,28.0,...,0.675054,0.617450,0.412512,0.617450,0.500975,0.181037,31.412344,70.115668,1.293828e+00,3.0


In [7]:
Working_condition_cols = ['Salary', 'Effort', 'Policy_influence', 'Teaching_percent',
                          'Research_percent', 'Academic_Resources', 'Perceived_Autonomy']


In [8]:
## Analyzing the bootstrap CI for the paths with age moderation:
i = 2

# Mediation_results = pd.DataFrame()
# Moderation_results = pd.DataFrame()
# Direct_path_results = pd.DataFrame()
for bootstrap_vals in [bootstrap_vals_pos_3, bootstrap_vals_pos_4, bootstrap_vals_pos_5, bootstrap_vals_pos_6]:
  i = i+1
  print(f'95% CI for Position {i}:')
  mean_results, lbound_results, ubound_results, pbci_results, pbci_90_results = bootstrap_analysis(bootstrap_vals, Working_condition_cols)
  pbci_results_transposed = pbci_results.set_index('Path').T
  # Mediation_results['Position '+str(i)] = pbci_results_transposed['Mediation']
  # Moderation_results['Position '+str(i)] = pbci_results_transposed['Moderation']
  # Direct_path_results['Position '+str(i)] = pbci_results_transposed['Direct Path']
  print(pbci_results_transposed)
  print('-'*50)

# Mediation_results

95% CI for Position 3:
Path                        Mediation         Moderation         Direct Path
Salary              [-0.0159, 0.0359]  [-0.0012, 0.0028]   [-0.0267, 0.1135]
Effort              [-0.0071, 0.0028]  [-0.0004, 0.0001]   [-0.0017, 0.0113]
Policy_influence    [-0.0505, 0.0549]  [-0.0033, 0.0054]   [-0.2053, 0.1001]
Teaching_percent    [-0.0028, 0.0055]  [-0.0002, 0.0004]     [-0.01, 0.0079]
Research_percent    [-0.0039, 0.0039]  [-0.0003, 0.0003]   [-0.0118, 0.0055]
Academic_Resources  [-0.0597, 0.0955]  [-0.0044, 0.0084]    [0.1287, 0.52] *
Perceived_Autonomy  [-0.0908, 0.0994]   [-0.009, 0.0049]  [0.1041, 0.5497] *
--------------------------------------------------
95% CI for Position 4:
Path                           Mediation         Moderation  \
Salary                 [-0.0035, 0.0051]  [-0.0007, 0.0003]   
Effort              [-0.0015, -0.0002] *     [-0.0, 0.0001]   
Policy_influence     [-0.0246, -0.003] *  [-0.0013, 0.0012]   
Teaching_percent       [-0.0013, 0.

In [9]:
## Analyzing the bootstrap CI for the paths without age moderation:
# Mediation_results_no_mod = pd.DataFrame()
# Direct_path_results_no_mod = pd.DataFrame()
i = 2
for bootstrap_vals_no_mod in [bootstrap_vals_no_mod_pos_3, bootstrap_vals_no_mod_pos_4,
                       bootstrap_vals_no_mod_pos_5, bootstrap_vals_no_mod_pos_6]:
  i = i+1
  print(f'95% CI for Position {i}:')
  mean_results, lbound_results, ubound_results, pbci_results, pbci_90_results = bootstrap_analysis(bootstrap_vals_no_mod, Working_condition_cols)
  pbci_results_transposed = pbci_results.set_index('Path').T
  # Mediation_results_no_mod['Position '+str(i)] = pbci_results_transposed['Mediation']
  # Direct_path_results_no_mod['Position '+str(i)] = pbci_results_transposed['Direct Path']
  print(pbci_results_transposed)
  print('-'*50)

# Mediation_results_no_mod


95% CI for Position 3:
Path                        Mediation         Direct Path
Salary              [-0.0071, 0.0135]   [-0.0267, 0.1135]
Effort              [-0.0028, 0.0011]   [-0.0017, 0.0113]
Policy_influence     [-0.042, 0.0184]   [-0.2054, 0.1001]
Teaching_percent     [-0.001, 0.0016]     [-0.01, 0.0079]
Research_percent     [-0.0018, 0.001]   [-0.0118, 0.0055]
Academic_Resources  [-0.0253, 0.0522]  [0.1288, 0.5197] *
Perceived_Autonomy  [-0.0225, 0.0695]   [0.104, 0.5497] *
--------------------------------------------------
95% CI for Position 4:
Path                           Mediation         Direct Path
Salary                 [-0.0023, 0.0062]  [0.0357, 0.0732] *
Effort              [-0.0018, -0.0004] *    [-0.0023, 0.002]
Policy_influence     [-0.0227, -0.002] *  [0.0203, 0.1042] *
Teaching_percent       [-0.0011, 0.0006]   [-0.0051, 0.0018]
Research_percent       [-0.0009, 0.0012]   [-0.0074, 0.0002]
Academic_Resources     [-0.0062, 0.0303]  [0.4627, 0.6163] *
Perceived_Au

In [10]:
for factor in bootstrap_vals_no_mod_pos_3.columns[2:9]:
  print('++'*40)
  print(factor)
  for bootstrap_vals_no_mod1 in [bootstrap_vals_no_mod_pos_3, bootstrap_vals_no_mod_pos_4,
                        bootstrap_vals_no_mod_pos_5, bootstrap_vals_no_mod_pos_6]:
    for bootstrap_vals_no_mod2 in [bootstrap_vals_no_mod_pos_3, bootstrap_vals_no_mod_pos_4,
                        bootstrap_vals_no_mod_pos_5, bootstrap_vals_no_mod_pos_6]:
      # print(bootstrap_vals_no_mod1['Position'].unique())
      position1 = bootstrap_vals_no_mod1['Position'].unique()
      position2 = bootstrap_vals_no_mod2['Position'].unique()
      if position1[0] < position2[0]:
        for path in ['Mediation', 'Direct Path']:
          print(f'Group 1: Position {position1[0]}')
          print(f'Group 2: Position {position2[0]}')
          data1 = bootstrap_vals_no_mod1[factor][bootstrap_vals_no_mod1['Path']==path].sort_values().reset_index(drop=True)
          data2 = bootstrap_vals_no_mod2[factor][bootstrap_vals_no_mod2['Path']==path].sort_values().reset_index(drop=True)
          data_diff = bootstrap_vals_no_mod2[factor][bootstrap_vals_no_mod2['Path']==path] - bootstrap_vals_no_mod1[factor][bootstrap_vals_no_mod1['Path']==path]
          data_diff = data_diff.sort_values().reset_index(drop=True)


          median1 = data1.median()
          median2 = data2.median()

          lbound_nr = math.ceil(len(data1)/2 - 1.96*np.sqrt(len(data1)/4))
          ubound_nr = math.floor(len(data1)/2 + 1.96*np.sqrt(len(data1)/4))

          lbound_val1 = data1[lbound_nr-1]
          ubound_val1 = data1[ubound_nr-1]

          lbound_val2 = data2[lbound_nr-1]
          ubound_val2 = data2[ubound_nr-1]

          lbound_diff = data_diff[lbound_nr-1]
          ubound_diff = data_diff[ubound_nr-1]

          print(f'Median CI for Position {position1[0]} - {path} ({factor}): [{lbound_val1:.4f}, {ubound_val1:.4f}]')
          print(f'Median CI for Position {position2[0]} - {path} ({factor}): [{lbound_val2:.4f}, {ubound_val2:.4f}]')
          print(f'Median CI for Difference - {path} ({factor}): [{lbound_diff:.4f}, {ubound_diff:.4f}]')
          print("*"*25)
  print('+-'*40)



++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Salary
Group 1: Position 3.0
Group 2: Position 4.0
Median CI for Position 3.0 - Mediation (Salary): [0.0008, 0.0011]
Median CI for Position 4.0 - Mediation (Salary): [0.0014, 0.0016]
Median CI for Difference - Mediation (Salary): [0.0000, 0.0005]
*************************
Group 1: Position 3.0
Group 2: Position 4.0
Median CI for Position 3.0 - Direct Path (Salary): [0.0472, 0.0506]
Median CI for Position 4.0 - Direct Path (Salary): [0.0533, 0.0543]
Median CI for Difference - Direct Path (Salary): [0.0026, 0.0061]
*************************
Group 1: Position 3.0
Group 2: Position 5.0
Median CI for Position 3.0 - Mediation (Salary): [0.0008, 0.0011]
Median CI for Position 5.0 - Mediation (Salary): [-0.0036, -0.0031]
Median CI for Difference - Mediation (Salary): [-0.0053, -0.0049]
*************************
Group 1: Position 3.0
Group 2: Position 5.0
Median CI for Position 3.0 - Direct Path (Salary): [0.0472, 

In [11]:
## Calculating with 2-sample t-test:
for factor in bootstrap_vals_no_mod_pos_3.columns[2:9]:
  print('++'*40)
  print(factor)
  for bootstrap_vals_no_mod1 in [bootstrap_vals_no_mod_pos_3, bootstrap_vals_no_mod_pos_4,
                        bootstrap_vals_no_mod_pos_5, bootstrap_vals_no_mod_pos_6]:
    for bootstrap_vals_no_mod2 in [bootstrap_vals_no_mod_pos_3, bootstrap_vals_no_mod_pos_4,
                        bootstrap_vals_no_mod_pos_5, bootstrap_vals_no_mod_pos_6]:
      # print(bootstrap_vals_no_mod1['Position'].unique())
      position1 = bootstrap_vals_no_mod1['Position'].unique()
      position2 = bootstrap_vals_no_mod2['Position'].unique()
      if position1[0] < position2[0]:
        for path in ['Mediation', 'Direct Path']:
          print(f'Group 1: Position {position1[0]} ({factor}) - {path}')
          print(f'Group 2: Position {position2[0]} ({factor}) - {path}')
          data1 = bootstrap_vals_no_mod1[factor][bootstrap_vals_no_mod1['Path']==path].sort_values().reset_index(drop=True)
          data2 = bootstrap_vals_no_mod2[factor][bootstrap_vals_no_mod2['Path']==path].sort_values().reset_index(drop=True)
          t_statistic, p_value = stats.ttest_ind(data1, data2)
          print(f"Statistic: {t_statistic}")
          print(f"P-value: {p_value}")
          if p_value <= 0.05:
            print('SIGNIFICANT')
          print('-'*50)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Salary
Group 1: Position 3.0 (Salary) - Mediation
Group 2: Position 4.0 (Salary) - Mediation
Statistic: 2.2294333736570717
P-value: 0.02584042607330558
SIGNIFICANT
--------------------------------------------------
Group 1: Position 3.0 (Salary) - Direct Path
Group 2: Position 4.0 (Salary) - Direct Path
Statistic: -7.429241935537475
P-value: 1.328124190298396e-13
SIGNIFICANT
--------------------------------------------------
Group 1: Position 3.0 (Salary) - Mediation
Group 2: Position 5.0 (Salary) - Mediation
Statistic: 41.760973511210615
P-value: 1.19981886e-316
SIGNIFICANT
--------------------------------------------------
Group 1: Position 3.0 (Salary) - Direct Path
Group 2: Position 5.0 (Salary) - Direct Path
Statistic: 34.90853421899561
P-value: 2.763693336520701e-233
SIGNIFICANT
--------------------------------------------------
Group 1: Position 3.0 (Salary) - Mediation
Group 2: Position 6.0 (Salary

In [12]:
for factor in bootstrap_vals_pos_3.columns[2:9]:
  print('++'*40)
  print(factor)
  for bootstrap_vals1 in [bootstrap_vals_pos_3, bootstrap_vals_pos_4,
                        bootstrap_vals_pos_5, bootstrap_vals_pos_6]:
    for bootstrap_vals2 in [bootstrap_vals_pos_3, bootstrap_vals_pos_4,
                        bootstrap_vals_pos_5, bootstrap_vals_pos_6]:
      position1 = bootstrap_vals1['Position'].unique()
      position2 = bootstrap_vals2['Position'].unique()
      if position1[0] < position2[0]:
        for path in ['Mediation', 'Moderation', 'Direct Path']:
          print(f'Group 1: Position {position1[0]} ({factor}) - {path}')
          print(f'Group 2: Position {position2[0]} ({factor}) - {path}')
          data1 = bootstrap_vals1[factor][bootstrap_vals1['Path']==path].sort_values().reset_index(drop=True)
          data2 = bootstrap_vals2[factor][bootstrap_vals2['Path']==path].sort_values().reset_index(drop=True)
          t_statistic, p_value = stats.ttest_ind(data1, data2)
          print(f"Statistic: {t_statistic}")
          print(f"P-value: {p_value}")
          if p_value <= 0.05:
            print('SIGNIFICANT')
          print('-'*50)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Salary
Group 1: Position 3.0 (Salary) - Mediation
Group 2: Position 4.0 (Salary) - Mediation
Statistic: 21.87219704551263
P-value: 2.910106871945407e-100
SIGNIFICANT
--------------------------------------------------
Group 1: Position 3.0 (Salary) - Moderation
Group 2: Position 4.0 (Salary) - Moderation
Statistic: 32.17335564984519
P-value: 3.582361420325788e-202
SIGNIFICANT
--------------------------------------------------
Group 1: Position 3.0 (Salary) - Direct Path
Group 2: Position 4.0 (Salary) - Direct Path
Statistic: -7.428298614450266
P-value: 1.3374982439885226e-13
SIGNIFICANT
--------------------------------------------------
Group 1: Position 3.0 (Salary) - Mediation
Group 2: Position 5.0 (Salary) - Mediation
Statistic: 38.15182252612685
P-value: 7.038751354780035e-272
SIGNIFICANT
--------------------------------------------------
Group 1: Position 3.0 (Salary) - Moderation
Group 2: Position 5.0

**Bayesian:**

In [17]:
path_to_bayesian = 'https://raw.githubusercontent.com/AndreiAf02/Job_Satisfaction_Burnout/refs/heads/main/Bayesian/CZ/'

## Path values without moderation:
bayesian_vals_no_mod_pos_3 = pd.read_csv(path_to_bayesian+'CZ_burnout_posterior_no_mod_position_3.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_no_mod_pos_4 = pd.read_csv(path_to_bayesian+'CZ_burnout_posterior_no_mod_position_4.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_no_mod_pos_5 = pd.read_csv(path_to_bayesian+'CZ_burnout_posterior_no_mod_position_5.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_no_mod_pos_6 = pd.read_csv(path_to_bayesian+'CZ_burnout_posterior_no_mod_position_6.0.csv', sep=",", header=0).iloc[:,1:]

## Path values with moderation:
bayesian_vals_pos_3 = pd.read_csv(path_to_bayesian+'CZ_burnout_posterior_position_3.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_pos_4 = pd.read_csv(path_to_bayesian+'CZ_burnout_posterior_position_4.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_pos_5 = pd.read_csv(path_to_bayesian+'CZ_burnout_posterior_position_5.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_pos_6 = pd.read_csv(path_to_bayesian+'CZ_burnout_posterior_position_6.0.csv', sep=",", header=0).iloc[:,1:]

In [18]:
factor_unique = ['Salary', 'Effort', 'Teaching',
              'Research', 'Policy', 'Acad Resources', 'Autonomy']

dir_path_unique = ['dp1', 'dp2', 'dp3', 'dp4', 'dp5', 'dp6', 'dp7']


In [24]:
i = 2
for bayesian_vals in [bayesian_vals_pos_3, bayesian_vals_pos_4, bayesian_vals_pos_5, bayesian_vals_pos_6]:
  i = i+1
  print(f'95% CrI for Position {i}:')
  path_ci, path_prob, path_ci_mean = Bayesian_analysis(bayesian_vals, factor_unique, dir_path_unique)
  print(path_ci.set_index('Factor'))
  print('-'*50)

95% CrI for Position 3:
                          Mediation           Moderation           Direct Path
Factor                                                                        
Salary          [-0.01305, 0.03214]  [-0.00095, 0.00240]   [-0.00883, 0.10953]
Effort          [-0.00745, 0.00338]  [-0.00038, 0.00017]   [-0.00276, 0.01162]
Teaching        [-0.00194, 0.00481]  [-0.00015, 0.00035]   [-0.01009, 0.00789]
Research        [-0.00280, 0.00290]  [-0.00018, 0.00022]   [-0.01403, 0.00620]
Policy          [-0.03148, 0.04810]  [-0.00258, 0.00521]   [-0.20787, 0.09970]
Acad Resources  [-0.03980, 0.09835]  [-0.00296, 0.00716]  [0.11512, 0.53974] *
Autonomy        [-0.06225, 0.05281]  [-0.00734, 0.00289]  [0.15289, 0.54533] *
--------------------------------------------------
95% CrI for Position 4:
                             Mediation            Moderation  \
Factor                                                         
Salary             [-0.00434, 0.00508]   [-0.00083, 0.00025]  

In [25]:
i = 2
for bayesian_vals in [bayesian_vals_pos_3, bayesian_vals_pos_4, bayesian_vals_pos_5, bayesian_vals_pos_6]:
  i = i+1
  print(f'95% CrI for Position {i}:')
  path_ci, path_prob, path_ci_mean = Bayesian_analysis(bayesian_vals, factor_unique, dir_path_unique)
  print(path_ci_mean.set_index('Factor'))
  print('-'*50)

95% CrI for Position 3:
                           Mediation            Moderation  \
Factor                                                       
Salary            [0.00597, 0.00695]    [0.00042, 0.00695]   
Effort          [-0.00185, -0.00162]  [-0.00009, -0.00162]   
Teaching          [0.00084, 0.00098]    [0.00006, 0.00098]   
Research          [0.00001, 0.00012]    [0.00001, 0.00012]   
Policy            [0.00332, 0.00494]    [0.00065, 0.00494]   
Acad Resources    [0.01497, 0.01790]    [0.00110, 0.01790]   
Autonomy        [-0.00344, -0.00116]  [-0.00125, -0.00116]   

                         Direct Path  
Factor                                
Salary            [0.05016, 0.05281]  
Effort            [0.00445, 0.00477]  
Teaching        [-0.00140, -0.00101]  
Research        [-0.00415, -0.00371]  
Policy          [-0.06124, -0.05444]  
Acad Resources  [0.32017, 0.32992] *  
Autonomy        [0.33927, 0.34810] *  
--------------------------------------------------
95% CrI for Pos

In [16]:
i = 2
for bayesian_vals_no_mod in [bayesian_vals_no_mod_pos_3, bayesian_vals_no_mod_pos_4,
                             bayesian_vals_no_mod_pos_5, bayesian_vals_no_mod_pos_6]:
  i = i+1
  print(f'95% CrI for Position {i}:')
  path_ci, path_prob = Bayesian_analysis_no_mod(bayesian_vals_no_mod, factor_unique, dir_path_unique)
  print(path_ci.set_index('Factor'))
  print('-'*50)

95% CrI for Position 3:
                          Mediation           Direct Path
Factor                                                   
Salary          [-0.00828, 0.01575]   [-0.01237, 0.11104]
Effort          [-0.00337, 0.00136]   [-0.00267, 0.01235]
Teaching        [-0.00129, 0.00218]   [-0.01010, 0.00767]
Research        [-0.00233, 0.00173]   [-0.01249, 0.00582]
Policy          [-0.04645, 0.01901]   [-0.20690, 0.09658]
Acad Resources  [-0.03334, 0.04298]  [0.09797, 0.53165] *
Autonomy        [-0.02519, 0.07364]  [0.13963, 0.54675] *
--------------------------------------------------
95% CrI for Position 4:
                             Mediation           Direct Path
Factor                                                      
Salary             [-0.00244, 0.00607]  [0.03740, 0.07102] *
Effort          [-0.00174, -0.00043] *   [-0.00231, 0.00207]
Teaching           [-0.00114, 0.00052]   [-0.00506, 0.00165]
Research           [-0.00084, 0.00109]   [-0.00742, 0.00039]
Policy       

## Austria:
**Bootstrap:**

In [26]:
path_to_bootstrap = 'https://raw.githubusercontent.com/AndreiAf02/Job_Satisfaction_Burnout/refs/heads/main/Bootstrap/AT/'

## Path values with no moderation:
bootstrap_vals_no_mod_pos_2 = pd.read_csv(path_to_bootstrap+'AT_bootstrap_path_values_no_mod_position_2.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_no_mod_pos_2['Position'] = 2.
bootstrap_vals_no_mod_pos_3 = pd.read_csv(path_to_bootstrap+'AT_bootstrap_path_values_no_mod_position_3.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_no_mod_pos_3['Position'] = 3.
bootstrap_vals_no_mod_pos_4 = pd.read_csv(path_to_bootstrap+'AT_bootstrap_path_values_no_mod_position_4.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_no_mod_pos_4['Position'] = 4.
bootstrap_vals_no_mod_pos_5 = pd.read_csv(path_to_bootstrap+'AT_bootstrap_path_values_no_mod_position_5.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_no_mod_pos_5['Position'] = 5.
bootstrap_vals_no_mod_pos_6 = pd.read_csv(path_to_bootstrap+'AT_bootstrap_path_values_no_mod_position_6.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_no_mod_pos_6['Position'] = 6.

## Path values with moderation:
bootstrap_vals_pos_2 = pd.read_csv(path_to_bootstrap+'AT_bootstrap_path_values_position_2.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_pos_2['Position'] = 2.
bootstrap_vals_pos_3 = pd.read_csv(path_to_bootstrap+'AT_bootstrap_path_values_position_3.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_pos_3['Position'] = 3.
bootstrap_vals_pos_4 = pd.read_csv(path_to_bootstrap+'AT_bootstrap_path_values_position_4.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_pos_4['Position'] = 4.
bootstrap_vals_pos_5 = pd.read_csv(path_to_bootstrap+'AT_bootstrap_path_values_position_5.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_pos_5['Position'] = 5.
bootstrap_vals_pos_6 = pd.read_csv(path_to_bootstrap+'AT_bootstrap_path_values_position_6.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_pos_6['Position'] = 6.




In [27]:
Working_condition_cols = ['Salary', 'Effort', 'Policy_influence', 'Teaching_percent',
                          'Research_percent', 'Academic_Resources', 'Perceived_Autonomy']


In [28]:
## Analyzing the bootstrap CI for the paths with age moderation:
i = 1
for bootstrap_vals in [bootstrap_vals_pos_2, bootstrap_vals_pos_3, bootstrap_vals_pos_4, bootstrap_vals_pos_5, bootstrap_vals_pos_6]:
  i = i+1
  print(f'95% CI for Position {i}:')
  mean_results, lbound_results, ubound_results, pbci_results, pbci_90_results = bootstrap_analysis(bootstrap_vals, Working_condition_cols)
  print(pbci_results.set_index('Path').T)
  print('-'*50)


95% CI for Position 2:
Path                        Mediation         Moderation           Direct Path
Salary              [-0.0207, 0.0329]   [-0.001, 0.0007]      [-0.015, 0.0145]
Effort              [-0.0046, 0.0023]  [-0.0001, 0.0001]  [-0.0076, -0.0001] *
Policy_influence    [-0.1555, 0.0582]  [-0.0019, 0.0054]      [-0.025, 0.1887]
Teaching_percent    [-0.0111, 0.0041]  [-0.0001, 0.0004]     [-0.0031, 0.0098]
Research_percent    [-0.0065, 0.0039]  [-0.0001, 0.0002]     [-0.0001, 0.0087]
Academic_Resources  [-0.1172, 0.1577]   [-0.005, 0.0039]    [0.2012, 0.4553] *
Perceived_Autonomy  [-0.0481, 0.2348]  [-0.0064, 0.0015]    [0.2805, 0.5122] *
--------------------------------------------------
95% CI for Position 3:
Path                        Mediation         Moderation         Direct Path
Salary              [-0.0147, 0.0618]  [-0.0014, 0.0005]   [-0.0027, 0.0325]
Effort              [-0.0111, 0.0044]  [-0.0002, 0.0003]   [-0.0062, 0.0076]
Policy_influence    [-0.2995, 0.3346]  [

In [29]:
## Analyzing the bootstrap CI for the paths without age moderation:
i = 1
for bootstrap_vals_no_mod in [bootstrap_vals_no_mod_pos_2, bootstrap_vals_no_mod_pos_3, bootstrap_vals_no_mod_pos_4,
                       bootstrap_vals_no_mod_pos_5, bootstrap_vals_no_mod_pos_6]:
  i = i+1
  print(f'95% CI for Position {i}:')
  mean_results, lbound_results, ubound_results, pbci_results, pbci_90_results = bootstrap_analysis(bootstrap_vals_no_mod, Working_condition_cols)
  print(pbci_results.set_index('Path').T)
  print('-'*50)

95% CI for Position 2:
Path                        Mediation           Direct Path
Salary              [-0.0018, 0.0044]      [-0.015, 0.0145]
Effort              [-0.0009, 0.0003]  [-0.0076, -0.0001] *
Policy_influence    [-0.0071, 0.0225]      [-0.025, 0.1888]
Teaching_percent    [-0.0007, 0.0009]     [-0.0031, 0.0098]
Research_percent    [-0.0005, 0.0006]     [-0.0001, 0.0087]
Academic_Resources  [-0.0144, 0.0204]     [0.201, 0.4552] *
Perceived_Autonomy    [-0.02, 0.0547]    [0.2805, 0.5121] *
--------------------------------------------------
95% CI for Position 3:
Path                         Mediation         Direct Path
Salary              [0.0001, 0.0138] *   [-0.0027, 0.0325]
Effort               [-0.0028, 0.0003]   [-0.0062, 0.0076]
Policy_influence     [-0.0034, 0.0776]   [-0.1634, 0.1678]
Teaching_percent       [-0.002, 0.002]    [-0.006, 0.0108]
Research_percent     [-0.0006, 0.0027]   [-0.0029, 0.0107]
Academic_Resources   [-0.0055, 0.0987]  [0.0591, 0.4107] *
Perceived_

In [30]:
for factor in bootstrap_vals_no_mod_pos_3.columns[2:9]:
  print('++'*40)
  print(factor)
  for bootstrap_vals_no_mod1 in [bootstrap_vals_no_mod_pos_2, bootstrap_vals_no_mod_pos_3, bootstrap_vals_no_mod_pos_4,
                        bootstrap_vals_no_mod_pos_5, bootstrap_vals_no_mod_pos_6]:
    for bootstrap_vals_no_mod2 in [bootstrap_vals_no_mod_pos_2, bootstrap_vals_no_mod_pos_3, bootstrap_vals_no_mod_pos_4,
                        bootstrap_vals_no_mod_pos_5, bootstrap_vals_no_mod_pos_6]:
      # print(bootstrap_vals_no_mod1['Position'].unique())
      position1 = bootstrap_vals_no_mod1['Position'].unique()
      position2 = bootstrap_vals_no_mod2['Position'].unique()
      if position1[0] < position2[0]:
        for path in ['Mediation', 'Direct Path']:
          print(f'Group 1: Position {position1[0]}')
          print(f'Group 2: Position {position2[0]}')
          data1 = bootstrap_vals_no_mod1[factor][bootstrap_vals_no_mod1['Path']==path].sort_values().reset_index(drop=True)
          data2 = bootstrap_vals_no_mod2[factor][bootstrap_vals_no_mod2['Path']==path].sort_values().reset_index(drop=True)
          data_diff = bootstrap_vals_no_mod2[factor][bootstrap_vals_no_mod2['Path']==path] - bootstrap_vals_no_mod1[factor][bootstrap_vals_no_mod1['Path']==path]
          data_diff = data_diff.sort_values().reset_index(drop=True)


          median1 = data1.median()
          median2 = data2.median()

          lbound_nr = math.ceil(len(data1)/2 - 1.96*np.sqrt(len(data1)/4))
          ubound_nr = math.floor(len(data1)/2 + 1.96*np.sqrt(len(data1)/4))

          lbound_val1 = data1[lbound_nr-1]
          ubound_val1 = data1[ubound_nr-1]

          lbound_val2 = data2[lbound_nr-1]
          ubound_val2 = data2[ubound_nr-1]

          lbound_diff = data_diff[lbound_nr-1]
          ubound_diff = data_diff[ubound_nr-1]

          print(f'Median CI for Position {position1[0]} - {path} ({factor}): [{lbound_val1:.4f}, {ubound_val1:.4f}]')
          print(f'Median CI for Position {position2[0]} - {path} ({factor}): [{lbound_val2:.4f}, {ubound_val2:.4f}]')
          print(f'Median CI for Difference - {path} ({factor}): [{lbound_diff:.4f}, {ubound_diff:.4f}]')
          print("*"*25)
  print('+-'*40)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Salary
Group 1: Position 2.0
Group 2: Position 3.0
Median CI for Position 2.0 - Mediation (Salary): [0.0005, 0.0006]
Median CI for Position 3.0 - Mediation (Salary): [0.0050, 0.0055]
Median CI for Difference - Mediation (Salary): [0.0043, 0.0047]
*************************
Group 1: Position 2.0
Group 2: Position 3.0
Median CI for Position 2.0 - Direct Path (Salary): [0.0030, 0.0037]
Median CI for Position 3.0 - Direct Path (Salary): [0.0140, 0.0148]
Median CI for Difference - Direct Path (Salary): [0.0108, 0.0119]
*************************
Group 1: Position 2.0
Group 2: Position 4.0
Median CI for Position 2.0 - Mediation (Salary): [0.0005, 0.0006]
Median CI for Position 4.0 - Mediation (Salary): [-0.0014, -0.0006]
Median CI for Difference - Mediation (Salary): [-0.0024, -0.0015]
*************************
Group 1: Position 2.0
Group 2: Position 4.0
Median CI for Position 2.0 - Direct Path (Salary): [0.0030, 

**Bayesian:**

In [45]:
path_to_bayesian = 'https://raw.githubusercontent.com/AndreiAf02/Job_Satisfaction_Burnout/refs/heads/main/Bayesian/AT/'

## Path values without moderation:
bayesian_vals_no_mod_pos_2 = pd.read_csv(path_to_bayesian+'AT_burnout_posterior_no_mod_position_2.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_no_mod_pos_3 = pd.read_csv(path_to_bayesian+'AT_burnout_posterior_no_mod_position_3.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_no_mod_pos_4 = pd.read_csv(path_to_bayesian+'AT_burnout_posterior_no_mod_position_4.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_no_mod_pos_5 = pd.read_csv(path_to_bayesian+'AT_burnout_posterior_no_mod_position_5.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_no_mod_pos_6 = pd.read_csv(path_to_bayesian+'AT_burnout_posterior_no_mod_position_6.0.csv', sep=",", header=0).iloc[:,1:]

## Path values with moderation:
bayesian_vals_pos_2 = pd.read_csv(path_to_bayesian+'AT_burnout_posterior_position_2.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_pos_3 = pd.read_csv(path_to_bayesian+'AT_burnout_posterior_position_3.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_pos_4 = pd.read_csv(path_to_bayesian+'AT_burnout_posterior_position_4.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_pos_5 = pd.read_csv(path_to_bayesian+'AT_burnout_posterior_position_5.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_pos_6 = pd.read_csv(path_to_bayesian+'AT_burnout_posterior_position_6.0.csv', sep=",", header=0).iloc[:,1:]

In [46]:
factor_unique = ['Salary', 'Effort', 'Teaching',
              'Research', 'Policy', 'Acad Resources', 'Autonomy']

dir_path_unique = ['dp1', 'dp2', 'dp3', 'dp4', 'dp5', 'dp6', 'dp7']


In [50]:
i = 1
for bayesian_vals in [bayesian_vals_pos_2, bayesian_vals_pos_3, bayesian_vals_pos_4, bayesian_vals_pos_5, bayesian_vals_pos_6]:
  i = i+1
  print(f'95% CrI for Position {i}:')
  path_ci, path_prob, path_ci_mean = Bayesian_analysis(bayesian_vals, factor_unique, dir_path_unique)
  print(path_ci.set_index('Factor'))
  print('-'*50)

95% CrI for Position 2:
                          Mediation           Moderation  \
Factor                                                     
Salary          [-0.01477, 0.03895]  [-0.00108, 0.00044]   
Effort          [-0.00573, 0.00199]  [-0.00006, 0.00016]   
Teaching        [-0.01026, 0.00363]  [-0.00012, 0.00036]   
Research        [-0.00622, 0.00350]  [-0.00011, 0.00021]   
Policy          [-0.13868, 0.05746]  [-0.00181, 0.00486]   
Acad Resources  [-0.06374, 0.14882]  [-0.00454, 0.00214]   
Autonomy        [-0.05623, 0.16515]  [-0.00402, 0.00179]   

                           Direct Path  
Factor                                  
Salary             [-0.01313, 0.01884]  
Effort          [-0.00769, -0.00010] *  
Teaching           [-0.00345, 0.00992]  
Research           [-0.00036, 0.00876]  
Policy             [-0.01810, 0.18570]  
Acad Resources    [0.19909, 0.46145] *  
Autonomy          [0.27929, 0.50396] *  
--------------------------------------------------
95% CrI for Pos

In [51]:
i = 1
for bayesian_vals in [bayesian_vals_pos_2, bayesian_vals_pos_3, bayesian_vals_pos_4, bayesian_vals_pos_5, bayesian_vals_pos_6]:
  i = i+1
  print(f'95% CrI for Position {i}:')
  path_ci, path_prob, path_ci_mean = Bayesian_analysis(bayesian_vals, factor_unique, dir_path_unique)
  print(path_ci_mean.set_index('Factor'))
  print('-'*50)

95% CrI for Position 2:
                           Mediation           Moderation  \
Factor                                                      
Salary            [0.00726, 0.00843]  [-0.00023, 0.00843]   
Effort          [-0.00119, -0.00102]  [0.00003, -0.00102]   
Teaching        [-0.00203, -0.00171]  [0.00006, -0.00171]   
Research        [-0.00077, -0.00057]  [0.00002, -0.00057]   
Policy          [-0.02473, -0.02061]  [0.00077, -0.02061]   
Acad Resources    [0.01683, 0.02129]  [-0.00061, 0.02129]   
Autonomy          [0.02866, 0.03343]  [-0.00056, 0.03343]   

                           Direct Path  
Factor                                  
Salary              [0.00284, 0.00356]  
Effort          [-0.00397, -0.00380] *  
Teaching            [0.00323, 0.00353]  
Research            [0.00408, 0.00428]  
Policy              [0.08230, 0.08689]  
Acad Resources    [0.32621, 0.33212] *  
Autonomy          [0.38995, 0.39492] *  
--------------------------------------------------
95% Cr

In [25]:
i = 1
for bayesian_vals_no_mod in [bayesian_vals_no_mod_pos_2, bayesian_vals_no_mod_pos_3, bayesian_vals_no_mod_pos_4,
                             bayesian_vals_no_mod_pos_5, bayesian_vals_no_mod_pos_6]:
  i = i+1
  print(f'95% CrI for Position {i}:')
  path_ci, path_prob = Bayesian_analysis_no_mod(bayesian_vals_no_mod, factor_unique, dir_path_unique)
  print(path_ci.set_index('Factor'))
  print('-'*50)

95% CrI for Position 2:
                          Mediation             Direct Path
Factor                                                     
Salary          [-0.00142, 0.00436]     [-0.01258, 0.01935]
Effort          [-0.00097, 0.00031]  [-0.00798, -0.00028] *
Teaching        [-0.00076, 0.00080]     [-0.00271, 0.00977]
Research        [-0.00063, 0.00050]     [-0.00045, 0.00846]
Policy          [-0.00798, 0.02187]     [-0.01520, 0.18916]
Acad Resources  [-0.01180, 0.02161]    [0.19556, 0.45909] *
Autonomy        [-0.02402, 0.06292]    [0.27814, 0.50197] *
--------------------------------------------------
95% CrI for Position 3:
                           Mediation           Direct Path
Factor                                                    
Salary          [0.00042, 0.01377] *   [-0.00432, 0.03595]
Effort           [-0.00321, 0.00039]   [-0.00540, 0.00718]
Teaching         [-0.00207, 0.00213]   [-0.00516, 0.00944]
Research         [-0.00095, 0.00243]   [-0.00184, 0.00941]
Policy 